In [1]:
import torch
import numpy as np
import pandas as pd
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image
from imutils import paths
from places365.detector_best import build_detect

In [2]:
import os
os.chdir(r"./places365")

In [3]:
# th architecture to use
arch = 'resnet18'

# load the pre-trained weights
model_file = '%s_places365.pth.tar' % arch
if not os.access(model_file, os.W_OK):
    weight_url = 'http://places2.csail.mit.edu/models_places365/' + model_file
    os.system('wget ' + weight_url)

model = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()


# load the image transformer
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# load the class label
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

In [24]:
'''New 2020-11-28'''
def f_detect(img_name): # new for direct view
    # th architecture to use
    arch = 'resnet18'

    # load the pre-trained weights
    model_file = '%s_places365.pth.tar' % arch
    if not os.access(model_file, os.W_OK):
        weight_url = 'http://places2.csail.mit.edu/models_places365/' + model_file
        os.system('wget ' + weight_url)

    model = models.__dict__[arch](num_classes=365)
    checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
    state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict)
    model.eval()


    # load the image transformer
    centre_crop = trn.Compose([
            trn.Resize((256,256)),
            trn.CenterCrop(224),
            trn.ToTensor(),
            trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # load the class label
    file_name = 'categories_places365.txt'
    if not os.access(file_name, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
        os.system('wget ' + synset_url)
    classes = list()
    with open(file_name) as class_file:
        for line in class_file:
            classes.append(line.strip().split(' ')[0][3:])
    classes = tuple(classes)
    
    # Load the building list
    df = pd.read_csv("../building_class_list.csv")
    buildings = list(df['class'])

    # Open the img
    img = Image.open(img_name)
    input_img = V(centre_crop(img).unsqueeze(0))

    # forward pass
    logit = model.forward(input_img)
    h_x = F.softmax(logit, 1).data.squeeze()
    prob_all, label_all = h_x.sort(0, True)

    # find the best label which is building (from the first two highest prob)
    prob_all = list(prob_all)
    label_all = list(label_all)
    best_prob = prob_all[0]
    best_label = label_all[0]
    count = 0
    while(best_label not in buildings and count<3):
        count += 1
        prob_all.remove(best_prob)
        label_all.remove(best_label)
        if not prob_all:
            break
        else:
            best_prob = prob_all[0]
            best_label = label_all[0]
        
    return best_prob,best_label

In [6]:
def return_num_img(img_path):
    arr = img_path.split("/")
    name = arr[len(arr)-1]
    arr = name.split("_")
    num = arr[len(arr)-1].split(".")[0]
    
    return num

In [7]:
# Load the test image
imgPaths = sorted(list(paths.list_images("./test_imgs/")))
path_to_save = "./test_imgs/"
for imgPath in imgPaths:
    num = return_num_img(imgPath)
    name = "gsv_4" + num + ".jpg"
    path_renamed = path_to_save + name
    img = Image.open(imgPath)
    img.save(path_renamed)

In [33]:
# Load the test image
imgPath = sorted(list(paths.list_images("./test_imgs")))
# Path to save the dataset
path_to_save = "./data/buildings/"
while(imgPath):
    print(imgPath.pop())

./test_imgs\gsv_1523.jpg
./test_imgs\gsv_1522.jpg
./test_imgs\gsv_1521.jpg
./test_imgs\gsv_1517.jpg
./test_imgs\gsv_1516.jpg
./test_imgs\gsv_1515.jpg
./test_imgs\gsv_1514.jpg
./test_imgs\gsv_1513.jpg
./test_imgs\gsv_1512.jpg
./test_imgs\gsv_1511.jpg
./test_imgs\gsv_1510.jpg
./test_imgs\gsv_1509.jpg
./test_imgs\gsv_1508.jpg
./test_imgs\gsv_1507.jpg
./test_imgs\gsv_1506.jpg
./test_imgs\gsv_1505.jpg
./test_imgs\gsv_1504.jpg
./test_imgs\gsv_1503.jpg


In [55]:
df_class

,class,names,idxes,detected_building,probility,reason_unsaved
0,garage/outdoor,\gsv_1523.jpg,157,0,0,no building detected
1,parking_garage/outdoor,\gsv_1522.jpg,256,0,0,prob less than threshold
2,manufactured_home,\gsv_1521.jpg,221,0,0,no building detected
3,driveway,\gsv_1517.jpg,127,0,0,no building detected
4,highway,\gsv_1516.jpg,175,0,0,no building detected
5,parking_lot,\gsv_1515.jpg,257,0,0,no building detected
6,highway,\gsv_1514.jpg,175,0,0,no building detected
7,industrial_area,\gsv_1513.jpg,192,0,0,no building detected
8,manufactured_home,\gsv_1512.jpg,221,0,0,no building detected
9,driveway,\gsv_1511.jpg,127,0,0,no building detected


In [52]:
# Load the test image
imgPath = sorted(list(paths.list_images("./test_imgs")))
# Path to save the dataset
path_to_save = "./test_imgs/save"

# Load the classes
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)


# Load the building list
df = pd.read_csv("../building_class_list.csv")
buildings = list(df['class'])

class_save = []
idxes = []
names = []
probility = []
reasons = []
save = []

while(imgPath):
    image = imgPath.pop()
    # Building detection
    try:
        prob,label = f_detect(image)
        print(prob,label)
    except:
        print('f_detect can not process this images')
        continue

    name = image.replace("./data/test_imgs","")
    names.append(name)
    class_save.append(classes[label])
    idxes.append(label)
    detect = 0

    img = Image.open(image)
    if label in buildings: # Check if one of the top two classes detected is building
        if prob > 0.06: # Filter out the image if the probability is less than 0.1
            print('Save: ', name)
            path_buildings = path_to_save + name.replace('./test_imgs','')
            img.save(path_buildings)
            detect = 1
            reason = "None"
        else: # write to csv - reason - prob less than threshold
            reason = "prob less than threshold"
            prob = 0.0
            print(reason)
    else: # write to csv - reason - no building detected
        prob = 0.0
        reason = "no building detected"
        print(reason)
        
    probility.append(prob)
    reasons.append(reason)
    save.append(detect)

df_class = pd.DataFrame(class_save,columns =['class'])
df_idxes = pd.DataFrame(idxes)
df_names = pd.DataFrame(names)
df_save = pd.DataFrame(save)
df_probility = pd.DataFrame(probility)
df_reasons = pd.DataFrame(reasons)
df_class['names'] = df_names
#df_class['img_index']
df_class['idxes'] = df_idxes
df_class['detected_building'] = df_save
df_class['probility'] = df_probility
df_class['reason_unsaved'] = df_reasons
df_class.to_csv('df_class.csv')

tensor(0.0433) tensor(157)
no building detected
tensor(0.0218) tensor(256)
prob less than threshold
tensor(0.0276) tensor(221)
no building detected
tensor(0.0589) tensor(127)
no building detected
tensor(0.0458) tensor(175)
no building detected
tensor(0.0575) tensor(257)
no building detected
tensor(0.1137) tensor(175)
no building detected
tensor(0.0672) tensor(192)
no building detected
tensor(0.1047) tensor(221)
no building detected
tensor(0.0262) tensor(127)
no building detected
tensor(0.0558) tensor(338)
no building detected
tensor(0.0524) tensor(152)
no building detected
tensor(0.0506) tensor(257)
no building detected
tensor(0.0260) tensor(221)
no building detected
tensor(0.0262) tensor(257)
no building detected
tensor(0.1004) tensor(127)
no building detected
tensor(0.1688) tensor(213)
Save:  ./test_imgs\gsv_1504.jpg
tensor(0.0432) tensor(221)
no building detected


In [13]:
imgPath

[]

In [37]:
df_names

""


In [54]:
'''2020-11-28'''
# Load the test image   
imgPath = sorted(list(paths.list_images("./test_imgs")))
# Path to save the dataset
path_to_save = "./test_imgs/save"

# Load the classes
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

# Load the building list
df = pd.read_csv("../building_class_list.csv")
buildings = list(df['class'])

class_save = []
idxes = []
names = []
probility = []
reasons = []
save = []

while(imgPath):
    image = imgPath.pop()
    # Building detection
    try:
        prob,label = f_detect(image)
    except:
        continue
    print(image)
    name = image.replace("./test_imgs","")# this one should be modified, if different name
    names.append(name)
    class_save.append(classes[label])
    idxes.append(label)
    detect = 0

    img = Image.open(image)
    if label in buildings: # Check if one of the top two classes detected is building
        if prob > 0.07: # Filter out the image if the probability is less than 0.1
            print('Save: ', name)
            path_buildings = path_to_save + name
            img.save(path_buildings)
            detect = 1
            reason = "None"
        else: # write to csv - reason - prob less than threshold
            prob = 0.0
            reason = "prob less than threshold"
            print(reason)
    else: # write to csv - reason - no building detected
        prob = 0.0
        reason = "no building detected"
        print(reason)
        
    probility.append(prob)
    reasons.append(reason)
    save.append(detect)

df_class = pd.DataFrame(class_save,columns =['class'])
df_idxes = pd.DataFrame(idxes)
df_names = pd.DataFrame(names)
df_save = pd.DataFrame(save)
df_probility = pd.DataFrame(probility)
df_reasons = pd.DataFrame(reasons)
df_class['names'] = df_names
#df_class['img_index']
df_class['idxes'] = df_idxes
df_class['detected_building'] = df_save
df_class['probility'] = df_probility
df_class['reason_unsaved'] = df_reasons
df_class.to_csv('df_class.csv')

./test_imgs\gsv_1523.jpg
no building detected
./test_imgs\gsv_1522.jpg
prob less than threshold
./test_imgs\gsv_1521.jpg
no building detected
./test_imgs\gsv_1517.jpg
no building detected
./test_imgs\gsv_1516.jpg
no building detected
./test_imgs\gsv_1515.jpg
no building detected
./test_imgs\gsv_1514.jpg
no building detected
./test_imgs\gsv_1513.jpg
no building detected
./test_imgs\gsv_1512.jpg
no building detected
./test_imgs\gsv_1511.jpg
no building detected
./test_imgs\gsv_1510.jpg
no building detected
./test_imgs\gsv_1509.jpg
no building detected
./test_imgs\gsv_1508.jpg
no building detected
./test_imgs\gsv_1507.jpg
no building detected
./test_imgs\gsv_1506.jpg
no building detected
./test_imgs\gsv_1505.jpg
no building detected
./test_imgs\gsv_1504.jpg
Save:  \gsv_1504.jpg
./test_imgs\gsv_1503.jpg
no building detected


In [50]:
df_class.to_csv

TypeError: 'NoneType' object is not subscriptable

In [19]:
# Load the data
df = pd.read_csv("data/stats_facade_detection.csv")
df.head()
df[['detected_building', 'save_after_greenSSD', 'save_after_original_ssd']] = df[['detected_building', 'save_after_greenSSD', 'save_after_original_ssd']].replace(to_replace = 0.0, value = False)
df[['detected_building', 'save_after_greenSSD', 'save_after_original_ssd']] = df[['detected_building', 'save_after_greenSSD', 'save_after_original_ssd']].replace(to_replace = 1.0, value = True)

,img_index,Building,Road,Heading,Errors,Download,reason_x,class,names,Building_idxes,detected_building,probility,reason_unsaved,name,save_after_greenSSD,reason_y,save_after_original_ssd
0,0,47.42829 8.63718,47.4282 8.63708,408.012788,0,True,None,residential_neighborhood,\img_0.png,283.0,0.0,0.000000,no building detected,NaN,NaN,NaN,NaN
1,1,47.43052 8.63819,47.43047 8.63831,292.619865,0,True,None,cottage,\img_1.png,107.0,1.0,0.110487,None,img_1.jpg,1.0,None,1.0
2,2,47.43063 8.63873,47.43071 8.63863,488.659808,0,True,None,house,\img_2.png,183.0,1.0,0.180126,None,img_2.jpg,1.0,None,1.0
3,3,47.43115 8.6394,47.43107 8.63947,318.814075,0,True,None,cottage,\img_3.png,107.0,1.0,0.176232,None,img_3.jpg,1.0,None,1.0
4,4,47.43126 8.63966,47.43117 8.63974,318.366461,0,True,None,chalet,\img_4.png,87.0,1.0,0.119446,None,img_4.jpg,0.0,too many greens,NaN


In [20]:
df[['detected_building', 'save_after_greenSSD', 'save_after_original_ssd']] = df[['detected_building', 'save_after_greenSSD', 'save_after_original_ssd']].replace(to_replace = 0.0, value = False)
df[['detected_building', 'save_after_greenSSD', 'save_after_original_ssd']] = df[['detected_building', 'save_after_greenSSD', 'save_after_original_ssd']].replace(to_replace = 1.0, value = True)


In [21]:
df.to_csv('detect_facade.csv')

In [22]:
df.save_after_greenSSD.value_counts()

True     2853
False     202
Name: save_after_greenSSD, dtype: int64

In [23]:
df.save_after_original_ssd.value_counts()

True     2481
False     225
Name: save_after_original_ssd, dtype: int64

In [28]:
temp = df[df['detected_building'] == True]
mean_1 = temp['probility'].mean()
mean_1

0.20587808915810132

In [29]:
temp = temp[temp['save_after_greenSSD']==True]
mean_2 = temp['probility'].mean()
mean_2

0.20542501091062032

In [30]:
temp = temp[temp['save_after_original_ssd']==True]
mean_3 = temp['probility'].mean()
mean_3

0.20609914460056397